In [168]:
import pandas as pd
import os
import numpy as np
import re
# import datetime
from itertools import compress
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.calibration import calibration_curve
from sklearn.svm import NuSVC, SVC
# import imblearn
from collections import Counter
from imblearn.over_sampling import SMOTE 
from sklearn.linear_model import LogisticRegression
# from sklearn.preprocessing import label_binarize

In [67]:
# pd.set_option('display.height', 1000)
# pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

In [68]:
ref_df = pd.read_csv('which_race.csv', header=0)
csv_list = os.listdir('clean_frames')
counter = 34
ref_df.track = ref_df.track.apply(lambda x: x[1:] if x[0]==' '  else x)

In [69]:
# ref_df.set_index(['year','track'], inplace=True)

In [70]:
full_df = pd.read_csv('full_data.csv')
# full_df = pd.read_csv('full_data_w_tyres.csv')

/Users/guido/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [71]:
full_df['progress_since_pit'] = full_df['laps_since_pit']/full_df['last_lap']

In [72]:
full_df.drop(full_df[full_df['stop_count'].isna()].index,inplace=True)

full_df.drop(full_df[full_df['race_id'].isna()].index,inplace=True)

full_df.drop(full_df[full_df['target'].isna()].index,inplace=True)

full_df.drop(full_df[full_df['tyres'].isna()].index,inplace=True)

In [73]:
full_df = full_df.merge(ref_df,how='left',on='race_id')

In [74]:
full_df.drop(full_df[full_df['track'].isna()].index,inplace=True)

In [75]:
full_df = pd.get_dummies(full_df,columns=['NO'],drop_first=True)

In [76]:
full_df = pd.get_dummies(full_df,columns=['track'],drop_first=True)

In [77]:
full_df = pd.get_dummies(full_df,columns=['tyres'],drop_first=True)

In [78]:
fil = [e.isdigit() for e in full_df.columns]

In [79]:
fil2 = [e[2:].isdigit() for e in full_df.columns]

In [80]:
fil3 = [e[:6]=='track_' for e in full_df.columns]

In [81]:
fil4 = [e[:6]=='tyres_' for e in full_df.columns]

In [82]:
trackz = list(compress(list(full_df.columns), fil3))

In [83]:
distances = list(compress(list(full_df.columns), fil))

In [84]:
tyrez = list(compress(list(full_df.columns), fil4))

In [85]:
N_dummies = list(compress(list(full_df.columns), fil2))
# full_df.columns[full_df.columns.map(lambda x: x[2:].isdigit())]

In [191]:
features = ['progress','race_id','TIME', 'cumtime','stop_count','d_stop','year','race_in_year','median_pit','progress_since_pit','2sec_ahead','post_pit_traffic']

In [87]:
full_df['2sec_ahead'] = full_df[distances].apply(lambda x: ((x>0)&(x<2)).sum(),axis=1)

In [189]:
full_df['post_pit_traffic'] = full_df[distances].apply(lambda x: ((x-full_df['median_pit']>-1)&(x-full_df['median_pit']<3)).sum(),axis=1)

/Users/guido/anaconda/lib/python3.6/site-packages/pandas/core/indexes/base.py:3755: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  return this.join(other, how=how, return_indexers=return_indexers)


In [88]:
# full_df.head()

In [89]:
#full_df[num_features].apply(lambda x: x.fillna(x.mean(),inplace=True),axis=0)
full_df[distances] = full_df[distances].fillna(35)

In [190]:
full_df.to_csv('ready_df.csv')

# full_df = pd.read_csv('ready_df.csv')

# full_df.head()

In [92]:
# full_df.head(30)
# full_df.columns

In [194]:
test = (full_df.race_in_year>18) &(full_df.year==2017);

In [195]:
test.mean()

0.03170936581268374

In [196]:
# test_df = full_df.loc[test]

In [197]:
# train_df = full_df.loc[-test]

In [198]:
features_df = full_df[features+trackz+N_dummies+tyrez]#+num_features]
target = full_df['target']

In [199]:
features_train = features_df[-test]
features_test = features_df[test]
target_train = target[-test].astype('int32')
target_test = target[test].astype('int32')

In [112]:
sv_class = SVC(class_weight='balanced')

In [200]:
log_class = LogisticRegression()

In [201]:
log_class.fit(features_train,target_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [202]:
print('test accuracy is',log_class.score(features_test,target_test))
print('training accuracy is',log_class.score(features_train,target_train))

test accuracy is 0.9757174392935982
training accuracy is 0.9649027687414155


In [203]:
log_mat = confusion_matrix(target_test,log_class.predict(features_test));
log_mat

array([[884,   0],
       [ 22,   0]])

In [211]:
confusion_matrix(target_test,np.array([log_class.predict_proba(features_test)[:,1]>.9999]).reshape(len(target_test),))

array([[884,   0],
       [ 22,   0]])

In [147]:
rf_class = RandomForestClassifier(n_jobs=-1, n_estimators=10, random_state=0)

In [163]:
sv_class.fit(features_train,target_train)

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [153]:
rf_class.fit(features_train,target_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [154]:
print('test accuracy is',rf_class.score(features_test,target_test))
print('training accuracy is',rf_class.score(features_train,target_train))

test accuracy is 0.8964669051878354
training accuracy is 0.9981474300977985


In [164]:
print('test accuracy is',sv_class.score(features_test,target_test))
print('training accuracy is',sv_class.score(features_train,target_train))

test accuracy is 0.9733899821109123
training accuracy is 0.9745810176209556


In [171]:
rf_class.predict(features_test).mean()

0.10308586762075134

In [165]:
sv_class.predict(features_test).mean()

0.0038014311270125225

In [166]:
1-target_test.mean()

0.9767441860465116

In [157]:
1-target_train.mean()

0.5

In [469]:
# rf_class.predict_proba(features_test)[:,1]#
# [rf_class.predict_proba(features_test)[:,1]>.2]#.shape

In [221]:
def TFPN(x):
    print (x[0,0],'TN')
    print (x[0,1],'FP')
    print (x[1,0],'FN')
    print (x[1,1],'TP')
    print ('Recall', x[1,1]/(x[1,1]+x[1,0]))
    print ('Precision', x[1,1]/(x[1,1]+x[0,1]))
    return None

In [222]:
TFPN(log_mat)

884 TN
0 FP
22 FN
0 TP
Recall 0.0
Precision nan


/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in long_scalars
  import sys


In [167]:
sv_mat = confusion_matrix(target_test,sv_class.predict(features_test));
sv_mat

array([[4352,   16],
       [ 103,    1]])

In [160]:
rf_mat = confusion_matrix(target_test,rf_class.predict(features_test));
rf_mat

array([[3958,  410],
       [  53,   51]])

In [255]:
cmat = confusion_matrix(target_test,np.array([rf_class.predict_proba(features_test)[:,1]>.5]).reshape(len(target_test),));
cmat

array([[668, 216],
       [  4,  18]])

In [256]:
TFPN(cmat)

668 TN
216 FP
4 FN
18 TP
Recall 0.8181818181818182
Precision 0.07692307692307693


In [162]:
TFPN(rf_mat)

3958 TN
410 FP
53 FN
51 TP


In [111]:
confusion_matrix(target_test,rf_class.predict(features_test))

array([[4368,    0],
       [ 103,    1]])

In [170]:
TFPN(sv_mat)
# confusion_matrix(target_test,np.array([rf_class.predict_proba(features_test)[:,1]>.2]).reshape(len(target_test),))

4352 TN
16 FP
103 FN
1 TP


In [435]:
calibration_curve(target_train, rf_class.predict_proba(features_train)[:,1], normalize=False, n_bins=10)

(array([0., 0., 0., 1., 1., 1., 1., 1.]),
 array([0.00772034, 0.14195122, 0.24      , 0.56736842, 0.65333333,
        0.73903226, 0.85428571, 0.9490625 ]))

In [436]:
features_df.shape

(6434, 60)

In [568]:
# ref_df.to_csv('which_race.csv',index=False)
# ref_df = pd.read_csv('which_race.csv', header=0)
ref_df.head()

,race_id,year,race_in_year,track
0,0,2016,0,Melbourne
1,1,2016,10,Budapest
2,2,2016,12,Spa-Francorchamps
3,3,2016,17,Austin
4,4,2016,18,Mexico City


In [613]:
target_test.shape

(4972,)

array([[4702,  130],
       [ 130,   10]])

In [209]:
full_df.shape

(34475, 46)

In [589]:
target_test.sum()

140

In [212]:
sm = SMOTE()
features_train_sm, target_train_sm = sm.fit_sample(features_train, target_train)

In [124]:
# test_df = full_df.loc[test]

In [125]:
# train_df = full_df.loc[-test]

In [213]:
rf_class.fit(features_train_sm,target_train_sm)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [214]:
print('test accuracy is',rf_class.score(features_test,target_test))
print('training accuracy is',rf_class.score(features_train,target_train))

test accuracy is 0.7571743929359823
training accuracy is 0.9965300368683583


In [249]:
rf_mat = confusion_matrix(target_test,rf_class.predict_proba(features_test)[:,1]>.79);
rf_mat

array([[743, 141],
       [  9,  13]])

In [250]:
TFPN(rf_mat)

743 TN
141 FP
9 FN
13 TP
Recall 0.5909090909090909
Precision 0.08441558441558442


In [253]:
(13+743)/((141+9)+(13+743))

0.8344370860927153